In [1]:
#Education.xls file is the first data set
import pandas as pd
file_name='All excel sheets\Education.xls'
xl = pd.ExcelFile(file_name)
print(xl.sheet_names)

['Education 1970 to 2016']


In [3]:
# The first three lines are comments
df= xl.parse(0,header=4)

#Columns 1, 2, and 5 are state, area name(US, State, County), and the rural-urban continuum codes, respectively
df1=df.iloc[:,[1,2,5]]

# The last four columns are the percentages of adults with less than high school diploma, high school diploma, some college
# or associate's degree, and Bachelor's or higher degrees
df2=df.iloc[:,-4:]

#Concat df1 & df2
df_concat=pd.concat([df1,df2],axis='columns')

# Remove rows associate with Puerto Rico because this project examines the 50 states and District of Columbia
df_concat = df_concat[df_concat.State != 'PR']

#print the first five rows
df_concat.head()

,State,Area name,2013 Rural-urban Continuum Code,"Percent of adults with less than a high school diploma, 2012-2016","Percent of adults with a high school diploma only, 2012-2016","Percent of adults completing some college or associate's degree, 2012-2016","Percent of adults with a bachelor's degree or higher, 2012-2016"
0,US,United States,NaN,13.021,27.531,29.133,30.315
1,AL,Alabama,NaN,15.209,30.956,29.825,24.010
2,AL,Autauga County,2.0,12.417,34.331,28.660,24.593
3,AL,Baldwin County,3.0,9.972,28.692,31.788,29.547
4,AL,Barbour County,6.0,26.236,34.927,25.969,12.868


In [222]:
#Print the dataframe information
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3204 entries, 0 to 3203
Data columns (total 7 columns):
State                                                                         3204 non-null object
Area name                                                                     3204 non-null object
2013 Rural-urban Continuum Code                                               3143 non-null float64
Percent of adults with less than a high school diploma, 2012-2016             3194 non-null float64
Percent of adults with a high school diploma only, 2012-2016                  3194 non-null float64
Percent of adults completing some college or associate's degree, 2012-2016    3194 non-null float64
Percent of adults with a bachelor's degree or higher, 2012-2016               3194 non-null float64
dtypes: float64(5), object(2)
memory usage: 200.2+ KB


In [223]:
# The info above shows "2013 Rural-urban Continuum Code" column has the most null objects (61 null elements). 
# Each of the four columns of education levels have ten null objects as well
#I print the rows (only State and Area name) with null values in any column to investigate them
print(df_concat.loc[df_concat.isnull().any(axis=1),['State','Area name']])

     State                                    Area name
0       US                                United States
1       AL                                      Alabama
69      AK                                       Alaska
70      AK                             Aleutian Islands
86      AK                           Kuskokwim Division
94      AK  Prince of Wales-Outer Ketchikan Census Area
97      AK           Skagway-Yakutat-Angoon Census Area
98      AK            Skagway-Hoonah-Angoon Census Area
100     AK                         Upper Yukon Division
103     AK              Wrangell-Petersburg Census Area
106     AZ                                      Arizona
122     AR                                     Arkansas
198     CA                                   California
257     CO                                     Colorado
322     CT                                  Connecticut
331     DE                                     Delaware
335     DC                         District of C

In [224]:
# As it can be seen, missing data is from either US and States, or some counties from Alaska. I had to
# remove data from US and States anyway since the focus of this study is on the counties. In addition, eliminating around 12 counties from 
# more than 3100 counties means loosing about %0.4 of the data set, and therefore, it will not significantly impact the results
df_education=df_concat.dropna()

# Columns are renamed for convenience
df_education=df_education.rename(columns={'State':'state','Area name':'county','2013 Rural-urban Continuum Code':'RUCC',
                             'Percent of adults with less than a high school diploma, 2012-2016':'less_than_high_school',
                            'Percent of adults with a high school diploma only, 2012-2016':'high_school_diploma',
                            'Percent of adults completing some college or associate\'s degree, 2012-2016':'college/associate_degree',
                            'Percent of adults with a bachelor\'s degree or higher, 2012-2016':'bachelors/higher'})

#print the first five rows
df_education.head()

,state,county,RUCC,less_than_high_school,high_school_diploma,college/associate_degree,bachelors/higher
2,AL,Autauga County,2,12.417,34.331,28.660,24.593
3,AL,Baldwin County,3,9.972,28.692,31.788,29.547
4,AL,Barbour County,6,26.236,34.927,25.969,12.868
5,AL,Bibb County,1,19.302,41.816,26.883,12.000
6,AL,Blount County,1,19.969,32.942,34.039,13.050


In [225]:
# Print the information of df_education again to make sure there is no more null object
print(df_education.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3142 entries, 2 to 3203
Data columns (total 7 columns):
state                       3142 non-null object
county                      3142 non-null object
RUCC                        3142 non-null float64
less_than_high_school       3142 non-null float64
high_school_diploma         3142 non-null float64
college/associate_degree    3142 non-null float64
bachelors/higher            3142 non-null float64
dtypes: float64(5), object(2)
memory usage: 196.4+ KB
None


In [226]:
# The rural-urban continuum codes (RUCC) must be integers from 1-9. All the unique values are printed to make sure they meet the expectation
print(' All rural-urban continuum code values are\n',df_education.RUCC.unique())

# check if any education percentage is less than 0 or more than 100
print('\nNumber of rows with at least one education percentage less than 0 or more than 100 is ',\
      sum([df_education[(df_education[column]>100) | (df_education[column]<0)].shape[0] for column in df_education.iloc[:,-4:].columns]))

# check if the sum of the education percentages is close enough to 100 for all rows
total=df_education.iloc[:,-4:].sum(axis=1)
print('\nNumber of rows with the sum of education percentages less than 99.9 or more than 100.1 is ',\
      total[(total<99.9) | (total>100.1)].shape[0])

# check if there is more than one row for any county
state_county=df_education.state+'-'+df_education.county
print('\nTop five counties with the most counts')
print(state_county.value_counts().head())
# If the top ones have only one count, it means there is no duplicate row for any county

 All rural-urban continuum code values are
 [2. 3. 6. 1. 9. 7. 8. 4. 5.]

Number of rows with at least one education percentage less than 0 or more than 100 is  0

Number of rows with the sum of education percentages less than 99.9 or more than 100.1 is  0

Top five counties with the most counts
IL-Hancock County      1
TX-Red River County    1
OH-Madison County      1
AL-Choctaw County      1
MS-Alcorn County       1
dtype: int64


In [227]:
# read the poverty file
file_name='All excel sheets\PovertyEstimates.xls'
xl = pd.ExcelFile(file_name)
print(xl.sheet_names)

['Poverty Data 2016', 'Variable Descriptions']


In [228]:
# read the first sheet, collect three columns and rename them
df_poverty=xl.parse(0,header=2)
df_poverty=df_poverty.loc[:,['State','Area_Name','PCTPOVALL_2016']].rename(columns={'State':'state','Area_Name':'county',
                                                                                    'PCTPOVALL_2016':'poverty'})
#Print the first five rows
df_poverty.head()

,state,county,poverty
0,US,United States,14
1,AL,Alabama,17
2,AL,Autauga County,14
3,AL,Baldwin County,12
4,AL,Barbour County,30


In [229]:
#The rows corresponding to states are of no use in this project and will be eliminated when poverty dataframe is merged with
#education dataframe so it's not necessary to remove them right now

# Print the dataframe information
df_poverty.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3194 entries, 0 to 3193
Data columns (total 3 columns):
state      3194 non-null object
county     3194 non-null object
poverty    3193 non-null float64
dtypes: float64(1), object(2)
memory usage: 99.8+ KB


In [230]:
# There is only one null cell in df_poverty, so we could just remove it
df_poverty.dropna(inplace=True)

# check if there is any poverty percentage less than 0 or more than 100
print('Number of rows with the poverty percentage less than 0 or more than 100 is ',\
      df_poverty[(df_poverty.poverty>100) | (df_poverty.poverty<0)].shape[0])

# check if there are duplicate rows for any county
state_county=df_poverty.state+'-'+df_poverty.county
print('\nTop five counties with the most counts')
print(state_county.value_counts().head())

Number of rows with the poverty percentage less than 0 or more than 100 is  0

Top five counties with the most counts
DC-District of Columbia    2
VT-Vermont                 1
GA-Clinch County           1
OH-Ohio                    1
LA-Avoyelles Parish        1
dtype: int64


In [231]:
# It shows there are two rows for District of Columbia. The duplicate is dropped
df_poverty.drop_duplicates(inplace=True)

# Merge education and poverty dataframes
df_merge_education_poverty=df_education.merge(df_poverty,on=['state','county'])

# Print the information of the new dataframe
df_merge_education_poverty.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3131 entries, 0 to 3130
Data columns (total 8 columns):
state                       3131 non-null object
county                      3131 non-null object
RUCC                        3131 non-null float64
less_than_high_school       3131 non-null float64
high_school_diploma         3131 non-null float64
college/associate_degree    3131 non-null float64
bachelors/higher            3131 non-null float64
poverty                     3131 non-null float64
dtypes: float64(6), object(2)
memory usage: 220.1+ KB


In [232]:
# In the merged file there are 3131 entries which is 11 less than 3142 entires of df_education. Since there are already more than 
# 3000 entries, 11 entries will not significantly change analysis results

In [233]:
# read the Unemployment sheet
file_name=r'All excel sheets\Unemployment.xls'
xl=pd.ExcelFile(file_name)
print(xl.sheet_names)

['Unemployment Med HH Inc', 'Variable Descriptions']


In [234]:
# read the first sheet, collect three columns and rename them
df_unemployment=xl.parse(0)
df_unemployment=df_unemployment[['State','Area_name','Unemployment_rate_2016']].rename(columns={'State':'state','Area_name':'county',
                                                                                       'Unemployment_rate_2016'
                                                                                      :'unemployment'})
#Print the first five rows
df_unemployment.head()

,state,county,unemployment
0,AL,Alabama,6.0
1,AL,"Autauga County, AL",5.3
2,AL,"Baldwin County, AL",5.4
3,AL,"Barbour County, AL",8.6
4,AL,"Bibb County, AL",6.6


In [235]:
#Remove the state name from county names
df_unemployment.county=df_unemployment.county.map(lambda x: x[:len(x)-4])

# The command above will remove the last four characters from name of states and District of Columbia.
# The rows corresponding to states are of no use in this project and will be eliminated when unemployment dataframe is merged with
# df_merge_education_poverty, but we need District of Columbia, and it must be corrected.
df_unemployment.loc[df_unemployment['county']=='District of Colu','county']='District of Columbia'

#Print the first five rows
df_unemployment.head()

,state,county,unemployment
0,AL,Ala,6.0
1,AL,Autauga County,5.3
2,AL,Baldwin County,5.4
3,AL,Barbour County,8.6
4,AL,Bibb County,6.6


In [236]:
#Print information
df_unemployment.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3274 entries, 0 to 3273
Data columns (total 3 columns):
state           3274 non-null object
county          3274 non-null object
unemployment    3271 non-null float64
dtypes: float64(1), object(2)
memory usage: 102.3+ KB


In [237]:
# There are three null unemployment rates
df_unemployment.dropna(inplace=True)

#Check if there is any duplicate
state_county=df_unemployment.state+'-'+df_unemployment.county
print('\nTop five counties with most counts')
print(state_county.value_counts().head())


Top five counties with most counts
DC-District of Columbia    2
TX-Menard County           1
GA-Clinch County           1
KY-Pendleton County        1
LA-Avoyelles Parish        1
dtype: int64


In [238]:
#Remove the duplicate
df_unemployment.drop_duplicates(inplace=True)

In [239]:
#Merge all three tables together
df_merge_three=df_merge_education_poverty.merge(df_unemployment,on=['state','county'])

#Print the info of the dataframe to make sure no data is missing
df_merge_three.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3122 entries, 0 to 3121
Data columns (total 9 columns):
state                       3122 non-null object
county                      3122 non-null object
RUCC                        3122 non-null float64
less_than_high_school       3122 non-null float64
high_school_diploma         3122 non-null float64
college/associate_degree    3122 non-null float64
bachelors/higher            3122 non-null float64
poverty                     3122 non-null float64
unemployment                3122 non-null float64
dtypes: float64(7), object(2)
memory usage: 243.9+ KB


In [240]:
# Again, in the merged file there are 3122 entries which is 9 less than 3131 entires of df_merge_education_poverty. 
# The overall number of the eliminated counties through proccessing the three tables are 32 counties (12 in education dataframe,
# 11 in unemployment dataframe, and 9 in poverty dataframe). It means around 1% of the entire counties (32/(3122+32)~0.01).
# This 1% data loss will not significantly impact the results of this project

In [241]:
# read the states and regions sheet
file_name=r'All excel sheets\us census bureau regions and divisions.csv'
df_regions=pd.read_csv(file_name)

#Print the first five rows
df_regions.head()

,State,State Code,Region,Division
0,Alaska,AK,West,Pacific
1,Alabama,AL,South,East South Central
2,Arkansas,AR,South,West South Central
3,Arizona,AZ,West,Mountain
4,California,CA,West,Pacific


In [245]:
# Rename the State Code column to state
df_regions.rename(columns={'State Code':'state','Region':'region'},inplace=True)

# Add Region column to df_merge_three
df_merge_all = df_merge_three.merge(df_regions.loc[:,['state','region']])

# Rearrange the columns so the Region column is located after the county name
cols = df_merge_all.columns.tolist()
cols=cols[0:2]+cols[-1:]+cols[2:9]
df_merge_all = df_merge_all[cols]

# Print the firsr five rows
df_merge_all.head()

,state,county,region,RUCC,less_than_high_school,high_school_diploma,college/associate_degree,bachelors/higher,poverty,unemployment
0,AL,Autauga County,South,2,12.417,34.331,28.660,24.593,14,5.3
1,AL,Baldwin County,South,3,9.972,28.692,31.788,29.547,12,5.4
2,AL,Barbour County,South,6,26.236,34.927,25.969,12.868,30,8.6
3,AL,Bibb County,South,1,19.302,41.816,26.883,12.000,20,6.6
4,AL,Blount County,South,1,19.969,32.942,34.039,13.050,14,5.5


In [246]:
# Save the wrangled data to a csv file
df_merge_all.to_csv('wrangled_data.csv',index=False)